# Customer Revenue Prediction

## PyTorch LSTM Model
*Machine Learning Nanodegree Program | Capstone Project*

---

In this notebook I will be creating a PyTorch LSTM model and compare it with the baseline model I created earlier.

### Overview:
- Reading the data
- Preparing the tensors for the PyTorch Model
- Initializing the LSTM model
- Training the model with the train dataset
- Validating the model using the val dataset
- Predict the revenue for customer in test dataset
- Visualizing the results
- Compare the results with the baseline model
- Saving the results to a csv 

First, import the relevant libraries into notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sagemaker
import boto3

from os import path
from sklearn.metrics import mean_squared_error

%matplotlib inline
pd.set_option('display.float_format', lambda x: '%.10f' % x)

In [2]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket = sagemaker_session.default_bucket()

prefix = 'sagemaker/capstone-project'

print(bucket)

sagemaker-us-east-1-201308845573


Set the various paths for the training, validation, test files and storing the baseline results

In [3]:
data_dir = '../datasets'

if not path.exists(data_dir):
    raise Exception('{} directory not found.'.format(data_dir))

train_file = '{}/{}'.format(data_dir, 'train.zip')
print('\nTrain file: {}'.format(train_file))

val_file = '{}/{}'.format(data_dir, 'val.zip')
print('\nValidation file: {}'.format(val_file))

pred_val_file = '{}/{}'.format(data_dir, 'lstm_pred_val.zip')
print('\nValidation Prediction file: {}'.format(pred_val_file))

test_file = '{}/{}'.format(data_dir, 'test.zip')
print('\nTest file: {}'.format(test_file))

pred_test_file = '{}/{}'.format(data_dir, 'lstm_pred_test.zip')
print('\nTest Prediction file: {}'.format(pred_test_file))

imp_features_file = '{}/{}'.format(data_dir, 'lstm_importances-01.png')
print('\nImportant Features file: {}'.format(imp_features_file))

input_s3_train_file = sagemaker_session.upload_data(path=train_file, bucket=bucket, key_prefix=prefix)
print('\nInput data S3 Train file: {}'.format(input_s3_train_file))

input_s3_val_file = sagemaker_session.upload_data(path=val_file, bucket=bucket, key_prefix=prefix)
print('\nInput data S3 Val file: {}'.format(input_s3_val_file))

input_s3_dir = 's3://{}/{}'.format(bucket, prefix)
print('\nInput data S3 directory: {}'.format(input_s3_dir))



Train file: ../datasets/train.zip

Validation file: ../datasets/val.zip

Validation Prediction file: ../datasets/lstm_pred_val.zip

Test file: ../datasets/test.zip

Test Prediction file: ../datasets/lstm_pred_test.zip

Important Features file: ../datasets/lstm_importances-01.png

Input data S3 Train file: s3://sagemaker-us-east-1-201308845573/sagemaker/capstone-project/train.zip

Input data S3 Val file: s3://sagemaker-us-east-1-201308845573/sagemaker/capstone-project/val.zip

Input data S3 directory: s3://sagemaker-us-east-1-201308845573/sagemaker/capstone-project


In [4]:
empty_check = []

for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('\nTest passed!')

pytorch-inference-2020-01-27-21-58-07-803/model.tar.gz
pytorch-training-2020-01-27-21-28-25-807/source/sourcedir.tar.gz
pytorch-training-2020-01-27-21-45-45-442/source/sourcedir.tar.gz
pytorch-training-2020-01-27-21-50-40-727/source/sourcedir.tar.gz
pytorch-training-2020-01-27-22-11-17-423/source/sourcedir.tar.gz
sagemaker/capstone-project/pytorch-training-2020-01-27-21-28-25-807/debug-output/training_job_end.ts
sagemaker/capstone-project/pytorch-training-2020-01-27-21-45-45-442/debug-output/training_job_end.ts
sagemaker/capstone-project/pytorch-training-2020-01-27-21-50-40-727/debug-output/training_job_end.ts
sagemaker/capstone-project/pytorch-training-2020-01-27-21-50-40-727/output/model.tar.gz
sagemaker/capstone-project/pytorch-training-2020-01-27-22-11-17-423/debug-output/training_job_end.ts
sagemaker/capstone-project/train.zip
sagemaker/capstone-project/val.zip

Test passed!


Method to load the dataset from the files

In [5]:
def load_data(zip_path):
    df = pd.read_csv(
        zip_path,
        dtype={'fullVisitorId': 'str'},
        compression='zip'
    )
    
    [rows, columns] = df.shape

    print('Loaded {} rows with {} columns from {}.'.format(
        rows, columns, zip_path
    ))
    
    return df

Load the train, validation and test datasets.

In [6]:
%%time

train_df = load_data(train_file)
val_df = load_data(val_file)
test_df = load_data(test_file)

print()

Loaded 765707 rows with 26 columns from ../datasets/train.zip.
Loaded 137946 rows with 26 columns from ../datasets/val.zip.
Loaded 804684 rows with 25 columns from ../datasets/test.zip.

CPU times: user 11.4 s, sys: 408 ms, total: 11.8 s
Wall time: 11.5 s


In [7]:
# remove this at the end

import torch

train_df.head()

train_y = np.log1p(train_df['totals.transactionRevenue'].values)
train_y = train_y[:764928]
print(len(train_y))
train_y = torch.from_numpy(train_y).float().squeeze()

train_X = train_df.drop(['totals.transactionRevenue', 'fullVisitorId'], axis=1).values
train_X = train_X[:764928]
train_X = torch.from_numpy(train_X).float()

train_X = train_X.reshape(train_X.shape[0], 1, train_X.shape[1])

train_df.head()

764928


,totals.transactionRevenue,fullVisitorId,channelGrouping,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,...,trafficSource.keyword,trafficSource.medium,trafficSource.source,trafficSource.referralPath,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,visitNumber,visitStartTime
0,0.0000000000,1131660440785968503,0.5714285714,0.2844827586,0.0000000000,0.0000000000,0.8695652174,0.3951781971,0.4000000000,0.9251101322,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,1.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0884048864
1,0.0000000000,377306020877927890,0.5714285714,0.3534482759,0.0000000000,0.0000000000,0.2608695652,0.5796645702,1.0000000000,0.0484581498,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,1.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0899785187
2,0.0000000000,3895546263509774583,0.5714285714,0.2844827586,0.0000000000,0.0000000000,0.8695652174,0.4968553459,0.6000000000,0.8149779736,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,1.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0895117291
3,0.0000000000,4763447161404445595,0.5714285714,0.6724137931,0.0000000000,0.0000000000,0.2173913043,0.5796645702,0.4000000000,0.4096916300,...,0.2980894083,0.8000000000,0.4168336673,1.0000000000,1.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0900122290
4,0.0000000000,27294437909732085,0.5714285714,0.2844827586,0.5000000000,1.0000000000,0.0000000000,0.5796645702,0.6000000000,0.9559471366,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0025380711,0.0881587000


In [14]:
from sagemaker.pytorch import PyTorch

output_path = 's3://{}/{}'.format(bucket, prefix)

estimator = PyTorch(
    entry_point='lstm_train.py',
    source_dir='../models/pytorch/',
    role=role,
    output_path=output_path,
    sagemaker_session=sagemaker_session,
    framework_version='1.2',
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    hyperparameters={
        'input_dim': 24,
        'n_layers': 2,
        'epochs': 10,
        'batch-size': 512
    }
)

In [15]:
estimator.fit({'train': input_s3_dir})

2020-01-27 22:37:55 Starting - Starting the training job...
2020-01-27 22:37:57 Starting - Launching requested ML instances.........
2020-01-27 22:39:28 Starting - Preparing the instances for training......
2020-01-27 22:40:39 Downloading - Downloading input data...
2020-01-27 22:41:12 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-01-27 22:41:36,890 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-01-27 22:41:36,894 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-27 22:41:36,907 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-01-27 22:41:36,909 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-01-27 22:41:37,189 sagemaker-containers INFO     Module lstm_train does not provide a setup.py. 
Generating setup.py
2020-01-27 2

Epoch: 3/10... Step: 1900... Loss: 1.639843... Val Loss: 2.041805
Epoch: 3/10... Step: 2000... Loss: 1.974677... Val Loss: 2.038175
Epoch: 3/10... Step: 2100... Loss: 1.848485... Val Loss: 2.035059
Epoch: 3/10... Step: 2200... Loss: 1.693117... Val Loss: 2.035150
Epoch: 4/10... Step: 2300... Loss: 1.856161... Val Loss: 2.044096
Epoch: 4/10... Step: 2400... Loss: 1.881738... Val Loss: 2.038237
Epoch: 4/10... Step: 2500... Loss: 2.045511... Val Loss: 2.042588
Epoch: 4/10... Step: 2600... Loss: 2.008865... Val Loss: 2.037701
Epoch: 4/10... Step: 2700... Loss: 2.084508... Val Loss: 2.042070
Epoch: 4/10... Step: 2800... Loss: 2.031914... Val Loss: 2.036846
Epoch: 4/10... Step: 2900... Loss: 1.908682... Val Loss: 2.037694
Epoch: 5/10... Step: 3000... Loss: 1.958229... Val Loss: 2.040109
Epoch: 5/10... Step: 3100... Loss: 2.092288... Val Loss: 2.034365
Epoch: 5/10... Step: 3200... Loss: 2.277801... Val Loss: 2.035222
Epoch: 5/10... Step: 3300... Loss: 1.714889... Val Loss: 2.041286
Epoch: 5/1

In [16]:
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(
    model_data=estimator.model_data,
    role=role,
    framework_version='1.2',
    entry_point='lstm_predict.py',
    source_dir='../models/pytorch'
)

predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-----------------!

In [17]:
def get_batches(dataset, size=1024):
    for i in range(0, len(dataset), size):  
        yield dataset[i:(i + size)]
        
def predict_batch(predictor, dataset):
    pred_arr = np.array([])
    
    for next_batch in get_batches(dataset):
        temp_pred = predictor.predict(next_batch)
        
        pred_arr = np.append(pred_arr, temp_pred)
    
    return pred_arr

In [18]:
val_id = val_df['fullVisitorId'].values
val_id = val_id[:137216]

val_y = val_df['totals.transactionRevenue'].values
val_y = val_y[:137216]


val_X = val_df.drop(['totals.transactionRevenue', 'fullVisitorId'], axis=1).values
val_X = val_X[:137216]

val_X = val_X.reshape(val_X.shape[0], 1, val_X.shape[1])

pred_val = predict_batch(predictor, val_X)

pred_val[pred_val < 0] = 0

pred_val_data = {
    'fullVisitorId': val_id,
    'transactionRevenue': val_y,
    'predictedRevenue': np.expm1(pred_val)
}

pred_val_df = pd.DataFrame(pred_val_data)

pred_val_df = pred_val_df.groupby('fullVisitorId')['transactionRevenue', 'predictedRevenue'].sum().reset_index()

pred_val_df.head()

,fullVisitorId,transactionRevenue,predictedRevenue
0,0000062267706107999,0.0000000000,0.0000000000
1,0000085059828173212,0.0000000000,0.0000000000
2,000026722803385797,0.0000000000,0.0000000000
3,0000436683523507380,0.0000000000,0.0000000000
4,0000450371054833295,0.0000000000,0.0000000000


In [19]:
rsme_val = np.sqrt(
    mean_squared_error(
        np.log1p(pred_val_df['transactionRevenue'].values),
        np.log1p(pred_val_df['predictedRevenue'].values)
    )
)

print('\nRSME for validation data set: {:.10f}\n'.format(rsme_val))


RSME for validation data set: 2.1623281175



In [ ]:
test_id = test_df['fullVisitorId'].values
test_X = test_df.drop(['fullVisitorId'], axis=1)

test_X = test_X.reshape(test_X.shape[0], 1, test_X.shape[1])

pred_val = predict_batch(predictor, val_X)

pred_test[pred_test < 0] = 0

pred_test_data = {
    'fullVisitorId': test_id,
    'predictedRevenue': np.expm1(pred_test)
}

pred_test_df = pd.DataFrame(pred_test_data)

pred_test_df = pred_test_df.groupby('fullVisitorId')['predictedRevenue'].sum().reset_index()

pred_test_df.head()

In [ ]:
pred_val_df.to_csv(pred_val_file, index=False, compression='zip')

pred_test_df.to_csv(pred_test_file, index=False, compression='zip')

In [ ]:
def delete_endpoint(predictor):
    try:
        predictor.delete_endpoint()
        print('Deleted {}'.format(predictor.endpoint))
    except: 
        print('Already deleted: {}'.format(predictor.endpoint))
        

delete_endpoint(predictor)

In [ ]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket)

bucket_to_delete.objects.all().delete()